# Documentação Técnica: Extração de Dados de Anatomia Patológica - Fleury

## Objetivo Principal
**Este notebook realiza a extração e classificação automática de informações clínicas relevantes em laudos de anatomia patológica de mama do Fleury.** Utilizando técnicas de processamento de linguagem natural (NLP) com modelos de linguagem avançados (LLMs), o notebook identifica descritores de malignidade, graus histológicos e nucleares, formação de túbulos, índices mitóticos e tipos histológicos em laudos médicos de anatomia patológica mamária.

## Tecnologias Utilizadas
- **OpenAI/Databricks LLM API**: Para processamento e extração de informações dos textos dos laudos
- **PySpark**: Framework principal para processamento distribuído de dados
- **MLflow**: Para registro de métricas, experimentos e monitoramento dos resultados de extração
- **Delta Lake**: Sistema de armazenamento para tabelas de destino
- **Pandas**: Manipulação de dataframes para processamento local
- **Expressões Regulares (re)**: Validação e extração de padrões específicos nos textos dos laudos
- **Octoops**: Monitoramento e alertas de falhas

## Fluxo de Trabalho/Etapas Principais
1. **Definição dos Parâmetros**: Configuração de tabelas, filtros e critérios de extração
2. **Consulta de Dados**: Extração de laudos recentes da base de dados do Fleury
3. **Processamento via LLM**:
   - Definição do prompt especializado para extração de informações
   - Processamento batch dos laudos via API de LLM
   - Extração estruturada das informações relevantes
4. **Validação dos Resultados**:
   - Comparação entre extração via regex e via LLM
   - Cálculo de métricas de precisão
   - Registro de métricas no MLflow
5. **Persistência dos Dados**: Salvamento dos dados processados em tabela Delta

## Dados Envolvidos
- **Fonte**: Tabela `refined.saude_preventiva.fleury_laudos`
- **Filtros**:
  - Linha de cuidado: "mama"
  - Sexo: "F" (feminino)
  - Siglas de exame: "ANATPATP", "CTPUNC", "FISHHER"
  - Laudos contendo "Topografia: mama"
- **Tabela de Destino**: `refined.saude_preventiva.fleury_laudos_mama_anatomia_patologica_v2`
- **Informações Extraídas**:
  - **Descritores de Malignidade**: carcinoma, invasivo, invasor, sarcoma, metástase, metastático, maligno, maligna, cdi, cli, cdis
  - **Grau Histológico**: 1, 2 ou 3
  - **Grau Nuclear**: 1, 2 ou 3
  - **Formação de Túbulos**: 1, 2 ou 3
  - **Índice Mitótico**: valor numérico
  - **Tipo Histológico**: classificação específica do tipo de carcinoma

## Resultados/Saídas Esperadas
- DataFrame enriquecido com os campos extraídos dos laudos
- Registro de métricas de qualidade da extração via MLflow
- Dados persistidos na tabela Delta `refined.saude_preventiva.fleury_laudos_mama_anatomia_patologica_v2`
- Alertas via Sentinel (Octoops) em caso de falhas ou ausência de dados

## Pré-requisitos
- Ambiente Databricks configurado
- Acesso ao endpoint LLM Databricks (`databricks-llama-4-maverick` ou `teste-maverick`)
- Permissões de acesso às tabelas de origem e destino
- Bibliotecas instaladas: openai, mlflow, pandas, tqdm, databricks-feature-store, octoops

## Considerações Importantes/Observações
- A extração via LLM é comparada com uma extração via regex (considerada pseudo-gold) para validação
- O threshold definido para aceitação da qualidade da extração é de 80%
- O modelo está otimizado para identificar termos específicos de malignidade e classificações dentro do contexto oncológico mamário
- Os resultados são registrados no experimento MLflow `/Shared/saude_preventiva_mama/experiments_fleury_anatomopatologico`
- A persistência utiliza estratégia de merge (upsert) para garantir a atualização de registros já existentes

# Extração de dados - Anatomo Patologico
Serão analisados os descritores de malignidade.
**Descritores de MALIGNIDADE:**
- carcinoma
- invasivo
- invasor
- sarcoma
- metástase
- metastático
- maligno
- maligna
- cdi, cli, cdis

Outros labels a serem extraídos:

- **Grau histológico:** será sempre um algarismo 1, 2 ou 3 (apenas três categorias). Para encontrar, basta procurar o primeiro algarismo numérico após o termo **"grau histológico"**.

- **Grau nuclear:** será sempre um algarismo 1, 2 ou 3 (apenas três categorias). Para encontrar, basta procurar o primeiro algarismo numérico após o termo **"grau nuclear"**.

- **Formação de túbulos:** será sempre um algarismo 1, 2 ou 3 (apenas três categorias). Para encontrar, basta procurar o primeiro algarismo numérico após o termo **"formação de túbulos"**.

- **Índice mitótico:** será sempre um algarismo 1, 2 ou 3 (apenas três categorias). Para encontrar, basta procurar o primeiro algarismo numérico após o termo **"mm2"**. Nesse caso, é melhor procurar o termo **"mm2"** ao invés de **"índice mitótico"**.

- **Labels de tipos histológicos:**
  - Carcinoma de mama ductal invasivo (CDI)/SOE
  - Carcinoma de mama ductal in situ
  - Carcinoma de mama lobular invasivo
  - Carcinoma de mama lobular
  - Carcinoma de mama papilífero
  - Carcinoma de mama metaplásico
  - Carcinoma de mama mucinoso
  - Carcinoma de mama tubular
  - Carcinoma de mama cístico adenoide
  - Carcinoma de mama medular
  - Carcinoma de mama micropapilar
  - Carcinoma de mama misto (ductal e lobular) invasivo

### Instalação de pacotes necessários
**Objetivo da Célula:** Instalar as bibliotecas necessárias para executar o notebook.

**Pacotes Instalados:**
- **openai**: Cliente para comunicação com a API OpenAI/Databricks LLM
- **databricks-feature-store**: Biblioteca para interagir com o Feature Store do Databricks
- **octoops**: Biblioteca para monitoramento e alertas

A instalação é feita usando o comando mágico `%pip`, que é específico para ambientes Jupyter/Databricks. O parâmetro `-q` (quiet) é usado em algumas instalações para reduzir a verbosidade da saída.

In [0]:
def salvar_excel(df, nome_arquivo):
    """ 
    Salva um DataFrame em um arquivo Excel.

    Args:
        df (pandas.DataFrame): DataFrame a ser salvo.
        nome_arquivo (str): Nome do arquivo Excel.
    """
    %pip install openpyxl
    import pandas as pd

    # After installing, save the DataFrame to Excel again
    df.to_excel(nome_arquivo, index=False)

In [0]:
%pip install openai
# %pip install tqdm -q
# %pip install pandarallel -q
%pip install databricks-feature-store -q
%pip install octoops

### Reinicialização do ambiente Python
**Objetivo da Célula:** Reiniciar o kernel Python para garantir que as bibliotecas recém-instaladas sejam carregadas corretamente.

Esta célula executa o comando `dbutils.library.restartPython()`, que é específico do ambiente Databricks. Este comando reinicia o interpretador Python, garantindo que todas as bibliotecas instaladas na célula anterior sejam devidamente carregadas no ambiente de execução. Isso é necessário porque, em ambientes Jupyter/Databricks, as bibliotecas instaladas durante a execução do notebook só ficam disponíveis após a reinicialização do kernel.

In [0]:
dbutils.library.restartPython()

### Importação de bibliotecas e configuração do ambiente
**Objetivo da Célula:** Importar todas as bibliotecas necessárias para o processamento de dados e desativar exibições automáticas do MLflow.

**Dependências:**
- Bibliotecas instaladas nas células anteriores

**Bibliotecas Importadas:**
- **re**: Para manipulação de expressões regulares
- **os, sys**: Manipulação do sistema e ambiente
- **json, time**: Processamento de JSON e controle de tempo
- **warnings**: Controle de mensagens de aviso
- **mlflow**: Rastreamento de experimentos e métricas
- **tqdm**: Barras de progresso para processos iterativos
- **pandas, numpy**: Manipulação e análise de dados
- **typing**: Anotações de tipo para melhor documentação do código
- **openai**: Cliente para API de LLM
- **dateutil.relativedelta**: Cálculos avançados de datas
- **pyspark.sql**: Componentes do Spark SQL
- **databricks.feature_store**: Cliente para Feature Store do Databricks

**Configurações Realizadas:**
- Desativa a exibição automática do MLflow no notebook com `mlflow.tracing.disable_notebook_display()`

**Variáveis/Objetos Criados:**
- Não cria variáveis persistentes além das importações

**Saída/Impacto:**
- Prepara o ambiente de execução com todas as dependências necessárias
- Desativa logs automáticos do MLflow que poderiam sobrecarregar a interface do notebook

In [0]:
import re
import os
import sys
import json
import time
import warnings
import mlflow
from tqdm import tqdm
import pandas as pd
import numpy as np
from typing import List, Any
import openai
from dateutil.relativedelta import relativedelta
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from databricks.feature_store import FeatureStoreClient

mlflow.tracing.disable_notebook_display()

# spark = SparkSession.builder.appName("LLM_Extractor").getOrCreate()

### Definição de tabelas e filtros para extração de dados
**Objetivo da Célula:** Configurar as variáveis de tabela de destino e definir os filtros SQL para seleção de dados recentes e relevantes.

**Variáveis/Objetos Criados:**
- `table_anatom`: String com o nome completo da tabela de destino (refined.saude_preventiva.fleury_laudos_mama_anatomia_patologica_v2)
- `where_clause`: String contendo cláusula SQL WHERE para selecionar apenas registros novos (com datestamp maior que o último processado)
- `filtro_extracao`: String contendo filtros para seleção de registros específicos de mama feminina

**Lógica Detalhada:**
- Define a tabela de destino para resultados de anatomia patológica
- Cria uma cláusula WHERE que seleciona apenas registros mais recentes que o último datestamp na tabela de destino (processamento incremental)
- Define filtros para extração que limitam os registros aos laudos:
  - Da linha de cuidado "mama"
  - De pacientes do sexo feminino (UPPER(sexo_cliente) = 'F')
  - Com siglas de exames específicas (ANATPATP, CTPUNC, FISHHER)
  - Contendo a expressão "Topografia: mama" no texto do laudo

**Saída/Impacto:**
- As variáveis definidas serão utilizadas posteriormente na consulta SQL para extrair dados relevantes de laudos de anatomia patológica mamária
- A abordagem de filtro por datestamp garante processamento incremental, evitando reprocessamento de registros já analisados

In [0]:
# filtros de extração
table_anatom = "refined.saude_preventiva.fleury_laudos_mama_anatomia_patologica_v2" 

where_clause = f"""
WHERE
    flr.`_datestamp` >= (
        SELECT MAX(anatom._datestamp)
        FROM {table_anatom} anatom
    )
    """

 
filtro_extracao = """
    WHERE
        linha_cuidado  = 'mama'
        AND UPPER(sexo_cliente) = 'F'
        AND sigla_exame IN ("ANATPATP", "CTPUNC", "FISHHER")
        AND laudo_tratado RLIKE '(?i)Topografia: mama'
    
"""

### Construção e execução da consulta SQL
**Objetivo da Célula:** Criar uma consulta SQL para extrair laudos relevantes e executá-la para obter os dados a serem processados pelo modelo LLM.

**Variáveis/Objetos Criados:**
- `query`: String contendo a consulta SQL completa com placeholders para os filtros definidos anteriormente
- `df_spk`: DataFrame Spark resultante da execução da consulta

**Lógica Detalhada:**
1. Define uma consulta SQL com Common Table Expression (CTE) chamada `base`
2. Seleciona campos relevantes dos laudos: identificadores, datas, informações do exame e o conteúdo do laudo
3. Aplica os filtros definidos anteriormente:
   - Processamento incremental via `where_clause` (registros mais recentes que o último processado)
   - Filtros específicos de mama via `filtro_extracao` (linha de cuidado, sexo, siglas de exame, conteúdo do laudo)
4. Executa a consulta usando `spark.sql()`, formatando a string para incluir os filtros
5. Exibe o resultado via `display(df_spk)`

**Tabelas/Colunas Utilizadas:**
- Tabela: `refined.saude_preventiva.fleury_laudos` (alias `flr`)
- Colunas principais:
  - Identificadores: `id_marca`, `id_unidade`, `id_cliente`, `id_ficha`, `ficha`, `id_item`, `id_subitem`, `id_exame`
  - Datas: `dth_pedido`, `dth_resultado`
  - Dados do exame: `sigla_exame`, `laudo_tratado`, `linha_cuidado`, `sexo_cliente`
  - Metadata: `_datestamp`

**Saída/Impacto:**
- Cria o DataFrame `df_spk` contendo os laudos de anatomia patológica mamária que serão processados
- Exibe o conteúdo do DataFrame na interface do notebook para visualização prévia dos dados

In [0]:
query = f"""
WITH 
base AS (
    SELECT
        flr.id_marca,
        flr.id_unidade,
        flr.id_cliente, 
        flr.id_ficha,
        flr.ficha,
        flr.id_item, 
        flr.id_subitem, 
        flr.id_exame, 
        flr.dth_pedido,
        flr.dth_resultado,
        flr.sigla_exame,
        flr.laudo_tratado,
        flr.linha_cuidado,
        flr.sexo_cliente,
        flr.`_datestamp`
    FROM refined.saude_preventiva.fleury_laudos flr 
    {where_clause} 
     
)
SELECT *
FROM base
{filtro_extracao}
"""
df_spk = spark.sql(query)
display(df_spk)

### Obtenção do token de acesso Databricks
**Objetivo da Célula:** Obter o token de autenticação do Databricks para ser utilizado nas chamadas à API do LLM.

**Variáveis/Objetos Criados:**
- `DATABRICKS_TOKEN`: Token de autenticação para acesso às APIs Databricks

**Lógica Detalhada:**
- Utiliza a API interna do Databricks para obter o token da sessão atual
- A construção condicional verifica se o contexto do notebook está disponível
- Se o contexto existir, obtém o token via API
- Se não existir, define o token como None

**Saída/Impacto:**
- O token será utilizado posteriormente para autenticar as chamadas ao serviço de LLM do Databricks
- Este método de obtenção de token é mais seguro que armazenar o token diretamente no código

In [0]:
DATABRICKS_TOKEN = (
    dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
    if dbutils.notebook.entry_point.getDbutils().notebook().getContext() is not None
    else None
)

### Definição do prompt para extração de informações via LLM
**Objetivo da Célula:** Criar uma função que gera o prompt especializado para extração de informações de laudos médicos através de um modelo de linguagem.

**Função Definida:**
- `prompt_laudo(laudo_texto: str) -> str`: Função que recebe o texto do laudo e retorna o prompt formatado

**Lógica Detalhada:**
1. A função recebe o texto do laudo como parâmetro
2. Constrói um prompt estruturado com instruções específicas para o modelo LLM
3. O prompt inclui:
   - O texto do laudo entre aspas triplas
   - Instruções detalhadas sobre quais informações extrair e como formatá-las
   - Especificações sobre os descritores de malignidade, graus histológicos, nucleares, etc.
   - Formato esperado da saída (dicionário Python)
4. Retorna o prompt formatado como uma string

**Critérios de Extração no Prompt:**
- **Descritores de malignidade**: Lista de termos específicos (carcinoma, invasivo, etc.)
- **Grau histológico**: Valor numérico (1, 2 ou 3)
- **Grau nuclear**: Valor numérico (1, 2 ou 3)
- **Formação de túbulos**: Valor numérico (1, 2 ou 3)
- **Índice mitótico**: Valor numérico após "mm2"
- **Tipo histológico**: Correspondência com uma lista de tipos específicos de carcinomas

**Saída Esperada:**
- Um dicionário Python com os campos extraídos ou "NÃO INFORMADO" quando a informação não está presente

**Saída/Impacto:**
- Define a função que será utilizada posteriormente para criar prompts personalizados para cada laudo
- A qualidade desta formatação de prompt é crucial para a precisão da extração de informações pelo LLM

In [0]:
def prompt_laudo_template() -> str:
  prompt = """A seguir está um laudo médico de mamografia. Se alguma informação não estiver presente no texto, retorne "NÃO INFORMADO". Sempre retorne apenas o dicionário Python.

  Laudo clínico:
  \"\"\"{laudo_texto}\"\"\"

  ### Critérios de extração:

  - **Descritores de malignidade**: retorne uma **lista** com os termos de malignidade encontrados no texto (case-insensitive). Se nenhum for encontrado, retorne lista vazia `[]`. Lista de termos: ['carcinoma', "invasivo", "invasor", "sarcoma", "metástase", "metastático", "maligno", "maligna", "cdi", "cli", "cdis"]

  - **Grau histológico**: retorne o valor numérico do grau histológico.

  - **Grau nuclear**: retorne o valor numérico do grau nuclear.

  - **Formação de túbulos**: retorne o valor numérico caso exista formação de túbulos.

  - **Índice mitótico**: retorne o valor numérico do score do índice mitótico que aparece após o mm2.

  - **Tipo histológico**: identifique e retorne a frase correspondente se algum dos seguintes for mencionado (case-insensitive, variações aceitas):
    - Carcinoma de mama ductal invasivo
    - Carcinoma de mama ductal in situ
    - Carcinoma de mama lobular invasivo
    - Carcinoma de mama lobular
    - Carcinoma de mama papilífero
    - Carcinoma de mama metaplásico
    - Carcinoma de mama mucinoso
    - Carcinoma de mama tubular
    - Carcinoma de mama cístico adenoide
    - Carcinoma de mama medular
    - Carcinoma de mama micropapilar
    - Carcinoma de mama misto (ductal e lobular) invasivo

  ### Saída esperada (dicionário Python válido):
  ```python
  {{
    "descritores_malignidade": ["termo1", "termo2", ...],
    "grau_histologico": número | "NÃO INFORMADO",
    "grau_nuclear": número | "NÃO INFORMADO",
    "formacao_tubulos": número | "NÃO INFORMADO",
    "indice_mitotico": número | "NÃO INFORMADO",
    "tipo_histologico": "texto correspondente ou 'NÃO INFORMADO'"
  }}

  """ 
  return prompt.strip()

### Funções para extração por RegEx e validação das respostas do LLM
**Objetivo da Célula:** Implementar um conjunto de funções que extraem informações dos laudos usando expressões regulares e comparam com as respostas do LLM.

**Funções Definidas:**
1. Funções de extração por RegEx:
   - `extrai_descritores`: Encontra descritores de malignidade no texto
   - `extrai_grau_histologico`: Extrai o grau histológico (1, 2 ou 3)
   - `extrai_grau_nuclear`: Extrai o grau nuclear (1, 2 ou 3)
   - `extrai_formacao_tubulos`: Extrai o valor de formação de túbulos (1, 2 ou 3)
   - `extrai_indice_mitotico`: Extrai o índice mitótico
   - `extrai_tipo_histologico`: Identifica o tipo histológico específico

2. Função de avaliação:
   - `avalia_extracao_sem_ground_truth`: Compara a extração do LLM com a extração por RegEx

**Lógica Detalhada:**
- `extrai_descritores`: Procura por cada termo da lista TERMS no texto usando regex case-insensitive
- `extrai_grau_histologico`, `extrai_grau_nuclear`, `extrai_formacao_tubulos`: Procuram por padrões específicos seguidos de um dígito
- `extrai_indice_mitotico`: Procura por um valor numérico próximo ao termo "mm2" ou "mitoses"
- `extrai_tipo_histologico`: Compara o texto com uma lista predefinida de tipos histológicos
- `avalia_extracao_sem_ground_truth`: 
  1. Gera um pseudo-gold standard usando as funções de extração por RegEx
  2. Compara campo a campo com as extrações do LLM
  3. Retorna um relatório detalhado de concordância

**Constantes Definidas:**
- `TERMS`: Lista de termos associados a malignidade
- `TIPOS`: Lista de padrões de tipos histológicos a serem buscados

**Saída/Impacto:**
- As funções serão utilizadas para avaliar a qualidade das extrações do modelo LLM
- A avaliação usa as extrações por RegEx como pseudo-gold standard
- Os resultados da comparação serão utilizados para métricas e monitoramento do desempenho do modelo

In [0]:
TERMS = ['carcinoma', 'invasivo', 'invasor', 'sarcoma', 
         'metástase', 'metastático', 'maligno', 'maligna', 
         'cdi', 'cli', 'cdis']

def extrai_descritores(txt):
    achados = set()
    for termo in TERMS:
        # insensível a maiúsculas e minúsculas, plenos caracteres
        if re.search(rf"\b{re.escape(termo)}\b", txt, flags=re.IGNORECASE):
            achados.add(termo.lower())
    return sorted(achados)  # lista em ordem alfabética

def extrai_grau_histologico(txt):
    # Captura algo como "Grau histológico: 2" ou "grau histológico 2"
    m = re.search(r"grau\s+histol[oó]gico\s*[:\-]?\s*(\d)", txt, flags=re.IGNORECASE)
    if m:
        return int(m.group(1))
    return None

def extrai_grau_nuclear(txt):
    m = re.search(r"grau\s+nuclear\s*[:\-]?\s*(\d)", txt, flags=re.IGNORECASE)
    return int(m.group(1)) if m else None

def extrai_formacao_tubulos(txt):
    m = re.search(r"forma[cç][aã]o\s+de\s+t[uú]bulos\s*[:\-]?\s*(\d)", txt, flags=re.IGNORECASE)
    return int(m.group(1)) if m else None

# def extrai_indice_mitotico(txt):
#     # Ex.: "índice mitótico 3/10 mm2" ou "mitótico: 2 mm2"
#     m = re.search(r"mit[oó]tico\s*[:\-]?\s*(\d+)\s*/?\s*\d*\s*mm2", txt, flags=re.IGNORECASE)
#     if m:
#         return int(m.group(1))
#     return None

def extrai_indice_mitotico(txt):
    # Ex.: "índice mitótico 3/10 mm2", "mitótico: 2 mm2", "Índice mitótico: 1 mitose / mm2", "Índice mitótico: 11 mitoses / mm2"
    m = re.search(r"mit[oó]tico\s*[:\-]?\s*(\d+)(?:\s*/\s*\d+)?\s*(?:mitoses?|mitose)?\s*/?\s*mm2", txt, flags=re.IGNORECASE)
    if m:
        return int(m.group(1))
    return None

TIPOS = [
  "carcinoma de mama ductal invasivo",
  "carcinoma de mama ductal in situ",
  "carcinoma de mama lobular invasivo",
  "carcinoma de mama lobular",
  "carcinoma de mama papilífero",
  "carcinoma de mama metapl[aá]sico",
  "carcinoma de mama mucinoso",
  "carcinoma de mama tubular",
  "carcinoma de mama c[ií]stico adenoide",
  "carcinoma de mama medular",
  "carcinoma de mama micropapilar",
  "carcinoma de mama misto (ductal e lobular) invasivo"
]

def extrai_tipo_histologico(txt):
    txt_lower = txt.lower()
    for tipo in TIPOS:
        # usar comparação simplificada, removendo acentos se quiser
        padrao = tipo.lower()
        if padrao in txt_lower:
            return tipo  # retorna exatamente a frase padronizada
    return None

def avalia_extracao_sem_ground_truth(laudo_texto, json_modelo):
    # 1. Gera pseudo-gold
    descrs_hei = extrai_descritores(laudo_texto)
    gr_hist_hei = extrai_grau_histologico(laudo_texto)
    gr_nuc_hei  = extrai_grau_nuclear(laudo_texto)
    form_tub_hei= extrai_formacao_tubulos(laudo_texto)
    ind_mit_hei = extrai_indice_mitotico(laudo_texto)
    tipo_histo_hei = extrai_tipo_histologico(laudo_texto)

    json_heu = {
        "descritores_malignidade": descrs_hei,
        "grau_histologico": gr_hist_hei if gr_hist_hei is not None else "NÃO INFORMADO",
        "grau_nuclear": gr_nuc_hei if gr_nuc_hei is not None else "NÃO INFORMADO",
        "formacao_tubulos": form_tub_hei if form_tub_hei is not None else "NÃO INFORMADO",
        "indice_mitotico": ind_mit_hei if ind_mit_hei is not None else "NÃO INFORMADO",
        "tipo_histologico": tipo_histo_hei if tipo_histo_hei is not None else "NÃO INFORMADO"
    }

    # 2. Prepara json_modelo – já é recebido do ChatGPT como dicionário Python

    # 3. Comparações campo a campo:
    comparacoes = {}

    # 3.1. Descritores de malignidade: compara igualdade exata (acertos ou não)
    val_heu_desc = set(json_heu["descritores_malignidade"])
    val_mod_desc = set(json_modelo.get("descritores_malignidade", []))
    acertou_desc = (val_heu_desc == val_mod_desc)
    comparacoes["descritores_malignidade"] = {
        "pseudo_gold": json_heu["descritores_malignidade"],
        "IA": json_modelo.get("descritores_malignidade", []),
        "acertou": acertou_desc
    }

    # 3.2. Para cada campo numérico ou de texto, basta verificar igualdade exata
    def compara_campo(nome):
        val_heu = json_heu[nome]
        val_mod = json_modelo.get(nome, "NÃO INFORMADO")
        acertou = (val_heu == val_mod)
        return {
            "pseudo_gold": val_heu,
            "IA": val_mod,
            "acertou": acertou
        }

    for campo in ["grau_histologico", "grau_nuclear", "formacao_tubulos", "indice_mitotico", "tipo_histologico"]:
        comparacoes[campo] = compara_campo(campo)

    return json_heu, comparacoes

### Função para agregação e análise de resultados
**Objetivo da Célula:** Implementar uma função que agrega os resultados das comparações entre LLM e regex para calcular métricas de precisão.

**Função Definida:**
- `agrega_resultados(lista_comparacoes)`: Calcula estatísticas agregadas sobre a precisão da extração por campo

**Lógica Detalhada:**
1. Recebe uma lista de resultados de comparações (geradas pela função `avalia_extracao_sem_ground_truth`)
2. Calcula para cada campo de interesse:
   - Número total de acertos
   - Taxa de acerto (acertos ÷ total)
3. Estrutura os resultados em um dicionário aninhado por campo
4. Usa um `Counter` para facilitar a contagem de acertos por campo

**Campos Analisados:**
- `descritores_malignidade`: Acurácia na extração da lista de termos de malignidade
- `grau_histologico`: Acurácia na extração do grau histológico
- `grau_nuclear`: Acurácia na extração do grau nuclear
- `formacao_tubulos`: Acurácia na extração dos valores de formação de túbulos
- `indice_mitotico`: Acurácia na extração do índice mitótico
- `tipo_histologico`: Acurácia na extração do tipo histológico

**Saída/Impacto:**
- Retorna um dicionário estruturado com estatísticas detalhadas sobre a precisão da extração
- Este dicionário será utilizado para registro de métricas no MLflow e para avaliação da qualidade do modelo
- As taxas de acerto servirão para determinar se o modelo atende ao threshold de qualidade estabelecido

In [0]:
from collections import Counter

def agrega_resultados(lista_comparacoes):
    total_laudos = len(lista_comparacoes)
    
    # Conta quantos acertos em descritores_malignidade
    acertos_descritores = 0
    
    # Conta acertos por campo numérico/textual
    acertos_campos = Counter()
    
    for comp in lista_comparacoes:
        # Para descritores_malignidade, só existe "acertou"
        if comp["descritores_malignidade"]["acertou"]:
            acertos_descritores += 1
        
        # Para cada campo numérico/textual
        for campo in [
            "grau_histologico", 
            "grau_nuclear", 
            "formacao_tubulos", 
            "indice_mitotico", 
            "tipo_histologico"
        ]:
            if comp[campo]["acertou"]:
                acertos_campos[campo] += 1

    resultado = {
        "descritores_malignidade": {
            "acertos": acertos_descritores,
            "total": total_laudos,
            "taxa_acerto": acertos_descritores / total_laudos if total_laudos > 0 else 0.0
        }
    }

    for campo in [
        "grau_histologico", 
        "grau_nuclear", 
        "formacao_tubulos", 
        "indice_mitotico", 
        "tipo_histologico"
    ]:
        acertou = acertos_campos[campo]
        resultado[campo] = {
            "acertos": acertou,
            "total": total_laudos,
            "taxa_acerto": acertou / total_laudos if total_laudos > 0 else 0.0
        }

    return resultado

### Processamento e avaliação dos laudos com o modelo LLM
**Objetivo da Célula:** Processar os laudos extraídos utilizando o modelo LLM, avaliar a qualidade das extrações e preparar os dados para persistência.

**Dependências:**
- DataFrames e funções definidas anteriormente
- Token de autenticação Databricks
- Cliente OpenAI configurado

**Lógica Detalhada:**
1. Verifica se há dados disponíveis para processamento (`df_spk.count() > 0`)
2. Inicializa o cliente OpenAI com o token Databricks e configuração do endpoint
3. Define o contexto do agente LLM como "médico oncologista especialista em laudos de mamografia"
4. Converte o DataFrame Spark para Pandas para processamento local (limitado a 15 registros para teste)
5. Processa os laudos em batch utilizando a função `batch_generate`
6. Limpa e converte as respostas do LLM para formato estruturado utilizando `limpar_e_converter`
7. Avalia a qualidade das extrações comparando com extrações via RegEx
8. Calcula métricas agregadas de precisão

**Variáveis/Objetos Criados:**
- `llm_client`: Cliente para comunicação com a API do LLM
- `descricao_agente`: Descrição do papel do LLM na análise
- `df_local`: DataFrame Pandas para processamento local
- `df_respostas`: DataFrame Spark com as respostas processadas
- `resultados`: Lista de comparações entre extrações via LLM e RegEx
- `json_metricas`: Estatísticas agregadas sobre a qualidade da extração

**Saída/Impacto:**
- Cria um DataFrame enriquecido com as informações extraídas dos laudos
- Exibe o DataFrame processado via `display(df_respostas)`
- Prepara os dados para registro de métricas no MLflow e persistência em tabela Delta

In [0]:
# CORREÇÃO -=========================

from pyspark.sql.types import StructType, StructField, StringType, ArrayType, IntegerType, BooleanType
from pyspark.sql.functions import from_json, col, expr, regexp_replace, lit, regexp_extract, coalesce, udf, when, size, array_sort
from pyspark.sql import SparkSession
import pandas as pd
import mlflow

# Iniciar sessão Spark
spark = SparkSession.builder.appName("DfPandasparaSpark").getOrCreate()

# Definir modelo Foundation Model
model = "databricks-llama-4-maverick"  # Ajustar conforme o modelo disponível

# Schema para o JSON de saída do LLM
llm_output_schema = StructType([
StructField("descritores_malignidade", ArrayType(StringType()), True),
StructField("grau_histologico", StringType(), True),
StructField("grau_nuclear", StringType(), True),
StructField("formacao_tubulos", StringType(), True),
StructField("indice_mitotico", StringType(), True),
StructField("tipo_histologico", StringType(), True)
])

# Criar UDFs para as funções heurísticas
@udf(ArrayType(StringType()))
def udf_extrai_descritores(txt):
    if txt is None:
        return []
    return extrai_descritores(txt)

@udf(StringType())
def udf_extrai_grau_histologico(txt):
    if txt is None:
        return "NÃO INFORMADO"
    resultado = extrai_grau_histologico(txt)
    return str(resultado) if resultado is not None else "NÃO INFORMADO"

@udf(StringType())
def udf_extrai_grau_nuclear(txt):
    if txt is None:
        return "NÃO INFORMADO"
    resultado = extrai_grau_nuclear(txt)
    return str(resultado) if resultado is not None else "NÃO INFORMADO"

@udf(StringType())
def udf_extrai_formacao_tubulos(txt):
    if txt is None:
        return "NÃO INFORMADO"
    resultado = extrai_formacao_tubulos(txt)
    return str(resultado) if resultado is not None else "NÃO INFORMADO"

@udf(StringType())
def udf_extrai_indice_mitotico(txt):
    if txt is None:
        return "NÃO INFORMADO"
    resultado = extrai_indice_mitotico(txt)
    return str(resultado) if resultado is not None else "NÃO INFORMADO"

@udf(StringType())
def udf_extrai_tipo_histologico(txt):
    if txt is None:
        return "NÃO INFORMADO"
    resultado = extrai_tipo_histologico(txt)
    return resultado if resultado is not None else "NÃO INFORMADO"

# Realizar extração
if df_spk.count() > 0:

    # Obter o template do prompt
    prompt_template = prompt_laudo_template()

    # 1. Criar prompts dinâmicos injetando cada laudo no template
    df_with_prompts = df_spk.withColumn(
        "prompt_formatado",
        regexp_replace(
            lit(prompt_template),
            r"\{laudo_texto\}",
            col("laudo_tratado")
        )
    )

    # 2. Enviar prompts para o modelo Foundation Model usando ai_query
    df_with_llm_response = df_with_prompts.withColumn(
        "resposta_llm_raw",
        expr(f"ai_query('{model}', prompt_formatado)")
    )

    # 3. Limpar e extrair JSON da resposta
    df_with_cleaned_response = df_with_llm_response.withColumn(
        "resposta_llm_limpa",
        regexp_replace(
            regexp_replace(col("resposta_llm_raw"), "```python", ""),
            "```", ""
        )
    )

    # 4. Parsear o JSON
    df_with_parsed_response = df_with_cleaned_response.withColumn(
        "resposta_llm_json",
        from_json(
            regexp_extract(col("resposta_llm_limpa"), r'(\{[^}]+\})', 1),
            llm_output_schema
        )
    )

    # 5. Expandir os campos do JSON (resultados do LLM)
    df_with_llm_fields = df_with_parsed_response.withColumn(
        "llm_descritores_malignidade",
        coalesce(col("resposta_llm_json.descritores_malignidade"), expr("array()"))
    ).withColumn(
        "llm_grau_histologico",
        coalesce(col("resposta_llm_json.grau_histologico"), lit("NÃO INFORMADO"))
    ).withColumn(
        "llm_grau_nuclear",
        coalesce(col("resposta_llm_json.grau_nuclear"), lit("NÃO INFORMADO"))
    ).withColumn(
        "llm_formacao_tubulos",
        coalesce(col("resposta_llm_json.formacao_tubulos"), lit("NÃO INFORMADO"))
    ).withColumn(
        "llm_indice_mitotico",
        coalesce(col("resposta_llm_json.indice_mitotico"), lit("NÃO INFORMADO"))
    ).withColumn(
        "llm_tipo_histologico",
        coalesce(col("resposta_llm_json.tipo_histologico"), lit("NÃO INFORMADO"))
    )

    # 6. Aplicar funções heurísticas (pseudo-gold)
    df_with_heuristics = df_with_llm_fields.withColumn(
        "heu_descritores_malignidade",
        udf_extrai_descritores(col("laudo_tratado"))
    ).withColumn(
        "heu_grau_histologico",
        udf_extrai_grau_histologico(col("laudo_tratado"))
    ).withColumn(
        "heu_grau_nuclear",
        udf_extrai_grau_nuclear(col("laudo_tratado"))
    ).withColumn(
        "heu_formacao_tubulos",
        udf_extrai_formacao_tubulos(col("laudo_tratado"))
    ).withColumn(
        "heu_indice_mitotico",
        udf_extrai_indice_mitotico(col("laudo_tratado"))
    ).withColumn(
        "heu_tipo_histologico",
        udf_extrai_tipo_histologico(col("laudo_tratado"))
    )

    # 7. Criar colunas de comparação (acertos)
    df_with_comparisons = df_with_heuristics.withColumn(
        "acertou_descritores",
        (array_sort(col("heu_descritores_malignidade")) == array_sort(col("llm_descritores_malignidade")))
    ).withColumn(
        "acertou_grau_histologico",
        (col("heu_grau_histologico") == col("llm_grau_histologico"))
    ).withColumn(
        "acertou_grau_nuclear",
        (col("heu_grau_nuclear") == col("llm_grau_nuclear"))
    ).withColumn(
        "acertou_formacao_tubulos",
        (col("heu_formacao_tubulos") == col("llm_formacao_tubulos"))
    ).withColumn(
        "acertou_indice_mitotico",
        (col("heu_indice_mitotico") == col("llm_indice_mitotico"))
    ).withColumn(
        "acertou_tipo_histologico",
        (col("heu_tipo_histologico") == col("llm_tipo_histologico"))
    )

    # 8. Selecionar colunas finais
    df_respostas = df_with_comparisons.select(
        "id_marca", "id_unidade", "id_cliente", "id_ficha", "ficha", 
        "id_item", "id_subitem", "id_exame", "sigla_exame", 
        "dth_pedido", "dth_resultado", "_datestamp", 
        "laudo_tratado", "linha_cuidado",
        "resposta_llm_raw",
        # Campos LLM
        col("llm_descritores_malignidade").alias("descritores_malignidade"),
        col("llm_grau_histologico").alias("grau_histologico"),
        col("llm_grau_nuclear").alias("grau_nuclear"),
        col("llm_formacao_tubulos").alias("formacao_tubulos"),
        col("llm_indice_mitotico").alias("indice_mitotico"),
        col("llm_tipo_histologico").alias("tipo_histologico"),
        # Campos Heurísticos (pseudo-gold)
        "heu_descritores_malignidade",
        "heu_grau_histologico",
        "heu_grau_nuclear",
        "heu_formacao_tubulos",
        "heu_indice_mitotico",
        "heu_tipo_histologico",
        # Campos de Comparação
        "acertou_descritores",
        "acertou_grau_histologico",
        "acertou_grau_nuclear",
        "acertou_formacao_tubulos",
        "acertou_indice_mitotico",
        "acertou_tipo_histologico"
    )

    display(df_respostas)

    # Definição de métricas regex vs llm
    lista_laudos = df_respostas.collect()
    resultados = []
    for row in lista_laudos:
        laudo_txt = row["laudo_tratado"]
        json_mod = {
            "descritores_malignidade": row["descritores_malignidade"] if row["descritores_malignidade"] else [],
            "grau_histologico": row["grau_histologico"],
            "grau_nuclear": row["grau_nuclear"],
            "formacao_tubulos": row["formacao_tubulos"],
            "indice_mitotico": row["indice_mitotico"],
            "tipo_histologico": row["tipo_histologico"]
        }
        pseudo_gold, compar = avalia_extracao_sem_ground_truth(laudo_txt, json_mod)
        resultados.append(compar)

    df_metrics = pd.DataFrame()
    df_metrics["laudos"] = df_respostas.select("laudo_tratado").toPandas()["laudo_tratado"]
    df_metrics["resultados"] = resultados
    resultados_expandidos = pd.json_normalize(df_metrics["resultados"])
    df_metrics = pd.concat(
        [df_metrics.drop(columns=["resultados"]), resultados_expandidos],
        axis=1
    )

    json_metricas = agrega_resultados(resultados)

In [0]:
# from pyspark.sql.types import StructType, StructField, StringType, ArrayType, IntegerType
# from pyspark.sql.functions import from_json, col, expr, regexp_replace, lit, regexp_extract, coalesce
# from pyspark.sql import SparkSession
# import pandas as pd
# import mlflow

# # Iniciar sessão Spark
# spark = SparkSession.builder.appName("DfPandasparaSpark").getOrCreate()

# # Definir modelo Foundation Model
# model = "databricks-llama-4-maverick"  # Ajustar conforme o modelo disponível

# # Schema para o JSON de saída do LLM
# llm_output_schema = StructType([
# StructField("descritores_malignidade", ArrayType(StringType()), True),
# StructField("grau_histologico", StringType(), True),
# StructField("grau_nuclear", StringType(), True),
# StructField("formacao_tubulos", StringType(), True),
# StructField("indice_mitotico", StringType(), True),
# StructField("tipo_histologico", StringType(), True)
# ])

# # Realizar extração
# if df_spk.count() > 0:

#     # Obter o template do prompt
#     prompt_template = prompt_laudo_template()

#     # 1. Criar prompts dinâmicos injetando cada laudo no template
#     df_with_prompts = df_spk.withColumn(
#         "prompt_formatado",
#         regexp_replace(
#             lit(prompt_template),
#             r"\{laudo_texto\}",
#             col("laudo_tratado")
#         )
#     )

#     # 2. Enviar prompts para o modelo Foundation Model usando ai_query
#     df_with_llm_response = df_with_prompts.withColumn(
#         "resposta_llm_raw",
#         expr(f"ai_query('{model}', prompt_formatado)")
#     )

#     # 3. Limpar e extrair JSON da resposta
#     df_with_cleaned_response = df_with_llm_response.withColumn(
#         "resposta_llm_limpa",
#         regexp_replace(
#             regexp_replace(col("resposta_llm_raw"), "```python", ""),
#             "```", ""
#         )
#     )

#     # 4. Parsear o JSON
#     df_with_parsed_response = df_with_cleaned_response.withColumn(
#         "resposta_llm_json",
#         from_json(
#             regexp_extract(col("resposta_llm_limpa"), r'(\{[^}]+\})', 1),
#             llm_output_schema
#         )
#     )

#     # 5. Expandir os campos do JSON
#     df_respostas = df_with_parsed_response.select(
#         "id_marca", "id_unidade", "id_cliente", "id_ficha", "ficha", 
#         "id_item", "id_subitem", "id_exame", "sigla_exame", 
#         "dth_pedido", "dth_resultado", "_datestamp", 
#         "laudo_tratado", "linha_cuidado",
#         "resposta_llm_raw",
#         coalesce(col("resposta_llm_json.descritores_malignidade"), expr("array()")).alias("descritores_malignidade"),
#         coalesce(col("resposta_llm_json.grau_histologico"), lit("NÃO INFORMADO")).alias("grau_histologico"),
#         coalesce(col("resposta_llm_json.grau_nuclear"), lit("NÃO INFORMADO")).alias("grau_nuclear"),
#         coalesce(col("resposta_llm_json.formacao_tubulos"), lit("NÃO INFORMADO")).alias("formacao_tubulos"),
#         coalesce(col("resposta_llm_json.indice_mitotico"), lit("NÃO INFORMADO")).alias("indice_mitotico"),
#         coalesce(col("resposta_llm_json.tipo_histologico"), lit("NÃO INFORMADO")).alias("tipo_histologico")
#     )

#     display(df_respostas)

#     # Definição de métricas regex vs llm
#     lista_laudos = df_respostas.collect()
#     resultados = []
#     for row in lista_laudos:
#         laudo_txt = row["laudo_tratado"]
#         json_mod = {
#             "descritores_malignidade": row["descritores_malignidade"] if row["descritores_malignidade"] else [],
#             "grau_histologico": row["grau_histologico"],
#             "grau_nuclear": row["grau_nuclear"],
#             "formacao_tubulos": row["formacao_tubulos"],
#             "indice_mitotico": row["indice_mitotico"],
#             "tipo_histologico": row["tipo_histologico"]
#         }
#         pseudo_gold, compar = avalia_extracao_sem_ground_truth(laudo_txt, json_mod)
#         resultados.append(compar)

#     df_metrics = pd.DataFrame()
#     df_metrics["laudos"] = df_respostas.select("laudo_tratado").toPandas()["laudo_tratado"]
#     df_metrics["resultados"] = resultados
#     resultados_expandidos = pd.json_normalize(df_metrics["resultados"])
#     df_metrics = pd.concat(
#         [df_metrics.drop(columns=["resultados"]), resultados_expandidos],
#         axis=1
#     )

#     json_metricas = agrega_resultados(resultados)

### Registro de métricas e experimentos no MLflow
**Objetivo da Célula:** Criar ou recuperar um experimento no MLflow e registrar as métricas de qualidade da extração.

**Dependências:**
- Variável `json_metricas` com resultados agregados da avaliação
- Bibliotecas mlflow e json

**Funções Definidas:**
- `get_or_create_experiment(experiment_name)`: Obtém ID de experimento existente ou cria um novo

**Lógica Detalhada:**
1. Define a função `get_or_create_experiment` para encontrar ou criar um experimento MLflow
2. Obtém o ID do experimento para saúde preventiva mamária
3. Configura o MLflow para registro automático de informações adicionais (`mlflow.autolog()`)
4. Define um threshold de qualidade (80%) para avaliar se as extrações são aceitáveis
5. Inicia uma nova execução (run) do MLflow
6. Para cada campo avaliado, registra:
   - Taxa de acerto (métrica principal)
   - Flag indicando se passou no threshold
   - Contagens absolutas de acertos e total
7. Registra o ID da execução para referência futura

**Parâmetros Registrados:**
- `modelo`: Nome do modelo LLM utilizado ("databricks-llama-4-maverick")

**Métricas Registradas:**
- Para cada campo (`descritores_malignidade`, `grau_histologico`, etc.):
  - `{campo}_taxa_acerto`: Porcentagem de extrações corretas
  - `{campo}_passou_threshold`: Flag binária (1 = passou, 0 = falhou)
  - `{campo}_acertos`: Número absoluto de extrações corretas
  - `{campo}_total`: Número total de documentos avaliados

**Saída/Impacto:**
- Cria um registro permanente da qualidade do modelo no sistema MLflow
- Permite comparação da performance entre diferentes versões do modelo
- Fornece ID de execução para referência e rastreabilidade

In [0]:
# import json
# # import requests
# import mlflow

# def get_or_create_experiment(experiment_name):
#     experiment = mlflow.get_experiment_by_name(experiment_name)
#     if experiment:
#         experiment_id = experiment.experiment_id
#     else:
#         experiment_id = mlflow.create_experiment(experiment_name)
#     mlflow.set_experiment(experiment_name)
#     return experiment_id


# # # criar experimento
# experiment_id = get_or_create_experiment("/Shared/saude_preventiva_mama/experiments_fleury_anatomopatologico")
# mlflow.autolog()

# threshold = 0.8

# with mlflow.start_run(experiment_id=experiment_id) as run:
#         #mlflow.log_param("modelo", payload["model"])
#         mlflow.log_param("modelo", "databricks-llama-4-maverick")
#         # mlflow.log_param("prompt_tokens", usage.prompt_tokens)
#         # mlflow.log_param("completion_tokens", usage.completion_tokens)
#         # mlflow.log_param("total_tokens",usage.total_tokens)
#         for campo, stats in json_metricas.items():
#             taxa = stats["taxa_acerto"]
#             mlflow.log_metric(f"{campo}_taxa_acerto", taxa)
#             passou_flag = 1 if taxa >= threshold else 0
#             mlflow.log_metric(f"{campo}_passou_threshold", passou_flag)
#             mlflow.log_metric(f"{campo}_acertos", stats["acertos"])
#             mlflow.log_metric(f"{campo}_total", stats["total"])
            
#             run_id = mlflow.active_run().info.run_id
#             print(f"Run registrada: {run_id}")

### Visualização das métricas de qualidade da extração
**Objetivo da Célula:** Exibir as métricas de qualidade da extração para análise visual.

Esta célula usa a função `display()` para mostrar o dicionário de métricas `json_metricas` que foi calculado anteriormente. Estas métricas fornecem uma visão detalhada do desempenho da extração de informações pelo modelo LLM em comparação com a extração por RegEx, incluindo taxas de acerto para cada campo extraído.

In [0]:
display(json_metricas)

### Persistência dos dados na tabela Delta
**Objetivo da Célula:** Salvar os dados processados na tabela Delta de destino usando uma estratégia de merge.

**Dependências:**
- DataFrame `df_respostas` contendo os dados processados
- Biblioteca Delta para operações de merge em tabelas
- Octoops (Sentinel) para monitoramento e alertas

**Funções Definidas:**
- `insert_data(df_spk, output_data_path)`: Realiza o merge dos dados na tabela Delta especificada

**Lógica Detalhada:**
1. Define a constante `WEBHOOK_DS_AI_BUSINESS_STG` como 'stg' (ambiente de staging)
2. Define a constante `OUTPUT_DATA_PATH` com o nome da tabela de destino
3. Implementa a função `insert_data`:
   - Carrega a tabela Delta existente
   - Realiza um merge usando o DataFrame como origem
   - Utiliza chaves de junção: ficha, id_item e id_subitem
   - Atualiza registros existentes e insere novos registros
4. Em um bloco try-except:
   - Verifica se o DataFrame tem registros (`df_respostas.count() > 0`)
   - Em caso positivo, chama `insert_data` para persistir os dados
   - Em caso negativo, envia alerta via Sentinel informando ausência de laudos para extração
   - Em caso de exceção, captura o erro, imprime o traceback e relança a exceção

**Saída/Impacto:**
- Os dados processados são salvos na tabela Delta usando estratégia de merge
- O sistema registra a quantidade de registros salvos
- Em caso de falha ou ausência de dados, um alerta é enviado via Sentinel para monitoramento
- Exceções são devidamente registradas e relançadas para o sistema de monitoramento

In [0]:
# from delta.tables import DeltaTable
# import traceback
# from octoops import Sentinel

# WEBHOOK_DS_AI_BUSINESS_STG = 'stg'

# OUTPUT_DATA_PATH = "refined.saude_preventiva.fleury_laudos_mama_anatomia_patologica_v2"

# # função para salvar dados na tabela
# def insert_data(df_spk, output_data_path):  
#     # Carrega a tabela Delta existente
#     delta_table = DeltaTable.forName(spark, output_data_path)

#     # Faz o merge (upsert)
#     (delta_table.alias("target")
#         .merge(
#             df_spk.alias("source"),
#             "target.ficha = source.ficha AND target.id_item = source.id_item AND target.id_subitem = source.id_subitem"
           
#         )
#         .whenMatchedUpdateAll() #atualiza todos os campos se o ID já existir
#         .whenNotMatchedInsertAll() #insere se o ID não existir
#         .execute())

# try:
#     if df_respostas.count() > 0:        
#         # Inserir tabela catalog
#     #    fs.write_table(
#     #         name="refined.saude_preventiva.fleury_laudos_mamo_anatomia_patologica",
#     #         df=df_final,
#     #         mode="merge",
#     #     )
#         insert_data(df_respostas, OUTPUT_DATA_PATH)
#         print('Total de registros salvos na tabela:', df_respostas.count())
#     else: 
#         error_message = traceback.format_exc()
#         error_message = "Fleury AnatomoPatologico - Não há laudos para extração."
#         sentinela_ds_ai_business = Sentinel(
#             project_name='Monitor_Linhas_Cuidado_Mama',
#             env_type=WEBHOOK_DS_AI_BUSINESS_STG,
#             task_title='Fleury AnatomoPatologico'
#         )

#         sentinela_ds_ai_business.alerta_sentinela(
#             categoria='Alerta', 
#             mensagem=error_message,
#             job_id_descritivo='3_fleury_mama_anatomopatologico'
#         )
# except Exception as e:
#     traceback.print_exc()
#     raise e

In [0]:
df2 = df_respostas.toPandas()
salvar_excel(df2, "resultados_anatomo_medicos6.xlsx")